In [1]:
!pip install python-Levenshtein

import numpy as np
import pandas as pd
import zipfile as zp
from pathlib import Path
from gensim.utils import check_output
from sklearn.preprocessing import normalize
from scipy.special import softmax
import shutil
from subprocess import check_output
import torch
from tqdm import tqdm
import colored

You should consider upgrading via the '/export/usuarios_ml4ds/lbartolome/topicmodeler/.venv/bin/python3 -m pip install --upgrade pip' command.


In [2]:
mallet_path = '/export/usuarios_ml4ds/jarenas/github/IntelComp/ITMT/topicmodeler/src/topicmodeling/mallet-2.0.8/bin/mallet'

In [3]:
modelsdir = Path("/export/usuarios_ml4ds/lbartolome/data/project_folder/TMmodels/Federated_test")

In [4]:
def printgr(text):
    print(colored.stylize(text, colored.fg('green')))

## **1. Creation of synthetic corpus**


---

We consider a scenario with n parties, each of them as an associated corpus.
To generate the corpus associated with each of the parties, we consider a common beta distribution (word-topic distribution), but we freeze different topics/ assign different asymmetric Dirichlet priors favoring different topics at the time of generating the document that composes each party's corpus.

We consider two scenarios for the last step (documents generation) according to whether we are utilzing LDA or ProdLDA's generative process.

* **According to LDA's generative process**
    **for** each document $d$ **do**  
    &nbsp;&nbsp;Draw topic distribution $\theta \sim Dirichlet(\alpha)$  
    &nbsp;&nbsp;&nbsp;&nbsp;**for** each word at position $n$ **do**  
    &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Sample topic $z_n \sim Multinomial(1,\theta)$  
    &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Sample word $w_n \sim Multinomial(1, \beta_{z_n})$  
    &nbsp;&nbsp;&nbsp;&nbsp;**end**  
    **end**

    where the distribution of $w_n \sim \beta$, $\theta$ is $Multinomial(1, \beta\theta)$

* **According to ProdLDA generative process**
    In comparisson to LDA's generative process:
    - $\beta$ is **unnormalized**
    - the conditional distribution of wn is defined as $w_n \mid \beta, \theta \sim Multinomial(1, \sigma(\beta\theta))$

### *Auxiliary functions for synthetic corpus generation*

In [5]:
def rotateArray(arr, n, d):
    temp = []
    i = 0
    while (i < d):
        temp.append(arr[i])
        i = i + 1
    i = 0
    while (d < n):
        arr[i] = arr[d]
        i = i + 1
        d = d + 1
    arr[:] = arr[: i] + temp
    return arr

In [6]:
def generateSynthetic(just_inf, vocab_size, n_topics, beta, alpha, n_docs,
                      n_docs_inf, nwords, alg, n_nodes, frozen_topics,
                      prior_frozen, own_topics, prior_nofrozen):
    
    if just_inf:
        n_total_docs = n_docs_inf
    else:
        n_total_docs = n_docs + n_docs_inf

    # Step 1 - generation of topics
    topic_vectors = np.random.dirichlet(vocab_size*[beta], n_topics)
    
    # Step 2 - generation of document topic proportions
    doc_topics_all = []
    for i in np.arange(n_nodes):
        doc_topics = np.random.dirichlet(prior_frozen + prior_nofrozen, n_total_docs)
        prior_nofrozen = rotateArray(prior_nofrozen, len(prior_nofrozen), own_topics)
        doc_topics_all.append(doc_topics)
        
    # Step 3 - Document generation
    documents_all = []
    z_all = []

    for i in np.arange(n_nodes):
        print("Generating document words for node ", str(i))
        documents = [] # Document words
        #z = [] # Assignments
        for docid in tqdm(np.arange(n_total_docs)):
            doc_len = np.random.randint(low=nwords[0], high=nwords[1])
            this_doc_words = []
            #this_doc_assigns = []
            for wd_idx in np.arange(doc_len):

                tpc = np.nonzero(np.random.multinomial(1, doc_topics_all[i][docid]))[0][0]
                #this_doc_assigns.append(tpc)
                if alg == "lda":
                    word = np.nonzero(np.random.multinomial(1, topic_vectors[tpc]))[0][0]
                else: #prodlda
                    pval = np.power(topic_vectors[tpc], doc_topics_all[i][docid][tpc])
                    weights = torch.tensor(pval, dtype=torch.float) # create a tensor of weights
                    word = torch.multinomial(weights, 1).numpy()[0]
                    #pval = normalize(pval[:,np.newaxis], norm='l1', axis=0).ravel()
                    #word = np.nonzero(np.random.multinomial(1, b))[0][0]
                this_doc_words.append('wd'+str(word))
            #z.append(this_doc_assigns)
            documents.append(this_doc_words)
        documents_all.append(documents)
        #z_all.append(z)
    
    return topic_vectors, doc_topics_all, documents_all

### *Topic modeling and Centralized settings*

In [7]:
# Topic modeling settings
vocab_size = 5000
n_topics = 50
beta = 1e-2
alpha = 50/n_topics#1/n_topics
n_docs = 10000
n_docs_inf = 10000
nwords = (150, 250) #Min and max lengths of the documents
alg = "lda" #"prod"

tm_settings = {
    "vocab_size": vocab_size,
    "n_topics": n_topics,
    "beta": beta,
    "alpha": alpha,
    "n_docs": n_docs,
    "n_docs_inf": n_docs_inf,
    "nwords": nwords,
    "alg": alg
}

In [8]:
# Centralized settings
n_nodes = 5
frozen_topics = 5
prior_frozen = frozen_topics * [alpha]
own_topics = int((n_topics-frozen_topics)/n_nodes)
prior_nofrozen = own_topics * [alpha] + (n_topics-frozen_topics-own_topics) * [alpha/10000]

centralized_settings = {
    "n_nodes": n_nodes,
    "frozen_topics": frozen_topics,
    "prior_frozen": prior_frozen,
    "own_topics": own_topics,
    "prior_nofrozen": prior_nofrozen
}

### *1.1. Generation of training + validation corpus (same generative process)*

In [9]:
topic_vectors, doc_topics_all, documents_all = generateSynthetic(False, **tm_settings, **centralized_settings)

Generating document words for node  0


100%|████████████████████████████████████████████████████████████████████████████| 20000/20000 [08:33<00:00, 38.91it/s]


Generating document words for node  1


100%|████████████████████████████████████████████████████████████████████████████| 20000/20000 [08:45<00:00, 38.05it/s]


Generating document words for node  2


100%|████████████████████████████████████████████████████████████████████████████| 20000/20000 [08:36<00:00, 38.72it/s]


Generating document words for node  3


100%|████████████████████████████████████████████████████████████████████████████| 20000/20000 [08:52<00:00, 37.58it/s]


Generating document words for node  4


100%|████████████████████████████████████████████████████████████████████████████| 20000/20000 [08:38<00:00, 38.56it/s]


### *1.2. Generation of second validation corpus (different generative process)*

In [10]:
topic_vectors2, doc_topics_all2, documents_all2 = generateSynthetic(True, **tm_settings, **centralized_settings)

Generating document words for node  0


100%|████████████████████████████████████████████████████████████████████████████| 10000/10000 [04:22<00:00, 38.03it/s]


Generating document words for node  1


100%|████████████████████████████████████████████████████████████████████████████| 10000/10000 [04:17<00:00, 38.84it/s]


Generating document words for node  2


100%|████████████████████████████████████████████████████████████████████████████| 10000/10000 [04:23<00:00, 38.01it/s]


Generating document words for node  3


100%|████████████████████████████████████████████████████████████████████████████| 10000/10000 [04:21<00:00, 38.17it/s]


Generating document words for node  4


100%|████████████████████████████████████████████████████████████████████████████| 10000/10000 [04:19<00:00, 38.56it/s]


## **2. Mallet**


---

### *Auxiliary functions*

In [11]:
def create_model_folder(modelname, modelsdir):
    
    # Create model folder and save model training configuration
    modeldir = modelsdir.joinpath(modelname)
    
    if modeldir.exists():

        # Remove current backup folder, if it exists
        old_model_dir = Path(str(modeldir) + '_old/')
        if old_model_dir.exists():
            shutil.rmtree(old_model_dir)

        # Copy current model folder to the backup folder.
        shutil.move(modeldir, old_model_dir)
        print(f'-- -- Creating backup of existing model in {old_model_dir}')

    modeldir.mkdir()
    configFile = modeldir.joinpath('trainconfig.json')
    
    return modeldir, configFile

In [12]:
def train(modelname, modelsdir, corpus):
    
    # Create model folder
    modeldir, configFile = create_model_folder(modelname, modelsdir)
    
    # Create corpus txt file
    corpusFile = modeldir.joinpath("corpus.txt")
    with open(corpusFile, 'w') as fout:
        [fout.write(str(idx) + ' 0 ' + ' '.join(doc) + '\n') for idx,doc in enumerate(corpus)]
    
    # Create corpus mallet file
    corpusMallet = modeldir.joinpath('corpus.mallet')

    cmd = mallet_path + \
        ' import-file --preserve-case --keep-sequence ' + \
        '--remove-stopwords --token-regex "' + '[\p{L}\p{N}][\p{L}\p{N}\p{P}]*' + \
        '" --input %s --output %s'
    cmd = cmd % (corpusFile, corpusMallet)

    try:
        print(f'-- -- Running command {cmd}')
        check_output(args=cmd, shell=True)
    except:
        print('-- -- Mallet failed to import data. Revise command')
    
    # Perform Mallet train
    with open(configFile, 'w', encoding='utf8') as fout:
        fout.write('input = ' + corpusMallet.resolve().as_posix() + '\n')
        fout.write('num-topics = ' + str(n_topics) + '\n')
        fout.write('alpha = 1\n')
        fout.write('optimize-interval = 10\n')
        fout.write('num-threads = 4\n')
        fout.write('num-iterations = 1000\n')
        fout.write('doc-topics-threshold = 0\n')
        fout.write('output-doc-topics = ' +
                    modeldir.joinpath('doc-topics.txt').resolve().as_posix() + '\n')
        fout.write('word-topic-counts-file = ' +
                   modeldir.joinpath('word-topic-counts.txt').resolve().as_posix() + '\n')
        fout.write('output-topic-keys = ' +
                   modeldir.joinpath('topickeys.txt').resolve().as_posix() + '\n')
        fout.write('inferencer-filename = ' + 
                   modeldir.joinpath('inferencer.mallet').resolve().as_posix() + '\n')
    cmd = mallet_path + ' train-topics --config ' + configFile.resolve().as_posix()
    print(cmd)
    try:
        print(
            f'-- -- Training mallet topic model. Command is {cmd}')
        check_output(args=cmd, shell=True)
    except:
        print('-- -- Model training failed. Revise command')
    
    return modeldir

In [13]:
def extractPipe(modeldir):
    
    path_corpus = modeldir.joinpath('corpus.mallet')
    if not path_corpus.is_file():
        print('-- Pipe extraction: Could not locate corpus file')
        return

    # Create auxiliary file with only first line from the original corpus file
    path_txt = modeldir.joinpath('corpus.txt')
    with path_txt.open('r', encoding='utf8') as f:
        first_line = f.readline()
        
    path_aux = modeldir.joinpath('corpus_aux.txt')
    with path_aux.open('w', encoding='utf8') as fout:
        fout.write(first_line + '\n')

    # We perform the import with the only goal to keep a small file containing the pipe
    print('-- Extracting pipeline')
    path_pipe = modeldir.joinpath('import.pipe')
    
    cmd = mallet_path + \
        ' import-file --use-pipe-from %s --input %s --output %s'   
    cmd = cmd % (path_corpus, path_aux, path_pipe)

    try:
        print(f'-- Running command {cmd}')
        check_output(args=cmd, shell=True)
    except:
        print('-- Failed to extract pipeline. Revise command')

    # Remove auxiliary file
    path_aux.unlink()

    return 

In [14]:
def inference(modeldir, corpus_inf):

    # A proper corpus should exist with the corresponding ipmortation pipe
    path_pipe = modeldir.joinpath('import.pipe')
    if not path_pipe.is_file():
        print('-- Inference error. Importation pipeline not found')
        return

    # Get inferencer
    inferencer = modeldir.joinpath('inferencer.mallet')
    
    # File for performing inference on
    corpus_file = modeldir.joinpath("corpus_inf.txt")
    with open(corpus_file, 'w') as fout:
        [fout.write(str(idx) + ' 0 ' + ' '.join(doc) + '\n') for idx,doc in enumerate(corpus_inf)]

    # The following files will be generated in the same folder
    corpus_mallet_inf = modeldir.joinpath('corpus_inf.mallet')  # mallet serialized
    doc_topics_file = modeldir.joinpath('doc-topics-inf.txt')

    # Import data to mallet
    print('-- Inference: Mallet Data Import')
   
    cmd = mallet_path + \
        ' import-file --use-pipe-from %s --input %s --output %s'
    cmd = cmd % (path_pipe, corpus_file, corpus_mallet_inf)

    try:
        print(f'-- Running command {cmd}')
        check_output(args=cmd, shell=True)
    except:
        print(
            '-- Mallet failed to import data. Revise command')
        return

    # Get topic proportions
    print('-- Inference: Inferring Topic Proportions')
    num_iterations = 100
    doc_topic_thr = 0

    cmd = mallet_path + \
        ' infer-topics --inferencer %s --input %s --output-doc-topics %s ' + \
        ' --doc-topics-threshold ' + str(doc_topic_thr) + \
        ' --num-iterations ' + str(num_iterations)
    cmd = cmd % (inferencer, corpus_mallet_inf, doc_topics_file)

    try:
        print(f'-- Running command {cmd}')
        check_output(args=cmd, shell=True)
    except:
        print('-- Mallet inference failed. Revise command')
        return

    return

In [15]:
def eval_betas(beta, topic_vectors):
    print('Tópicos (equivalentes) evaluados correctamente:')
    printgr(np.sum(np.max(np.sqrt(beta).dot(np.sqrt(topic_vectors.T)), axis=0)))

In [16]:
def eval_thetas(thetas_theoretical, thetas_actual, n_docs):
    sim_mat_theoretical = np.sqrt(thetas_theoretical).dot(np.sqrt(thetas_theoretical.T))
    sim_mat_actual = np.sqrt(thetas).dot(np.sqrt(thetas.T))
    print('Difference in evaluation of doc similarity:')
    printgr(np.sum(np.abs(sim_mat_theoretical - sim_mat_actual))/n_docs)

### *2.1. Centralized approach*

In [17]:
# Define corpus
corpus = [doc for docs_node in documents_all for doc in docs_node[0:n_docs]]
print(len(corpus))

# Train model 
modelname = "mallet_centralized"
modeldir = train(modelname, modelsdir, corpus)

50000
-- -- Creating backup of existing model in /export/usuarios_ml4ds/lbartolome/data/project_folder/TMmodels/Federated_test/mallet_centralized_old
-- -- Running command /export/usuarios_ml4ds/jarenas/github/IntelComp/ITMT/topicmodeler/src/topicmodeling/mallet-2.0.8/bin/mallet import-file --preserve-case --keep-sequence --remove-stopwords --token-regex "[\p{L}\p{N}][\p{L}\p{N}\p{P}]*" --input /export/usuarios_ml4ds/lbartolome/data/project_folder/TMmodels/Federated_test/mallet_centralized/corpus.txt --output /export/usuarios_ml4ds/lbartolome/data/project_folder/TMmodels/Federated_test/mallet_centralized/corpus.mallet
/export/usuarios_ml4ds/jarenas/github/IntelComp/ITMT/topicmodeler/src/topicmodeling/mallet-2.0.8/bin/mallet train-topics --config /export/usuarios_ml4ds/lbartolome/data/project_folder/TMmodels/Federated_test/mallet_centralized/trainconfig.json
-- -- Training mallet topic model. Command is /export/usuarios_ml4ds/jarenas/github/IntelComp/ITMT/topicmodeler/src/topicmodeling/

Mallet LDA: 50 topics, 6 topic bits, 111111 topic mask
Data loaded.
max tokens: 249
total tokens: 9981419
<10> LL/token: -8.12311
<20> LL/token: -7.81336
<30> LL/token: -7.70414
<40> LL/token: -7.62567

0	0.02	wd3301 wd4569 wd4837 wd4351 wd3663 wd2636 wd4939 wd2931 wd406 wd2543 wd1962 wd3055 wd862 wd417 wd1835 wd3380 wd4360 wd2971 wd1831 wd2011 
1	0.02	wd1619 wd1368 wd4763 wd2115 wd4980 wd3994 wd238 wd1736 wd4235 wd806 wd1278 wd1621 wd3221 wd3614 wd3053 wd4861 wd4118 wd1712 wd1790 wd73 
2	0.02	wd2289 wd233 wd3105 wd4457 wd1562 wd800 wd353 wd1640 wd3191 wd1820 wd2139 wd1172 wd1401 wd230 wd2236 wd422 wd3687 wd3555 wd4714 wd4931 
3	0.02	wd1023 wd4832 wd4442 wd4712 wd2771 wd2940 wd2438 wd1783 wd4098 wd1020 wd1900 wd3160 wd3557 wd4916 wd3377 wd784 wd2304 wd4743 wd1694 wd1085 
4	0.02	wd1735 wd230 wd112 wd220 wd1267 wd1428 wd4452 wd4830 wd4078 wd2305 wd3427 wd3994 wd2918 wd920 wd3000 wd3495 wd3420 wd1782 wd77 wd4397 
5	0.02	wd3321 wd1023 wd4777 wd2265 wd1412 wd4832 wd2578 wd354 wd4712 wd4442 

In [18]:
# Get betas
beta = np.zeros((tm_settings["n_topics"], tm_settings["vocab_size"]))
with open(modeldir.joinpath('word-topic-counts.txt').resolve().as_posix(), 'r', encoding='utf8') as fin:
    for line in fin.readlines():
        tokens = line.split()[1:]
        pos = int(tokens[0][2:])
        for el in tokens[1:]:
            tpc = int(el.split(':')[0])
            cnt = int(el.split(':')[1])
            beta[tpc,pos] = cnt

beta = normalize(beta,axis=1,norm='l1')

# Get thetas
thetas = np.loadtxt(modeldir.joinpath('doc-topics.txt').resolve().as_posix(), delimiter='\t', dtype=np.float32)[:,2:][:n_docs,:]
printgr(thetas.shape)
thetas[thetas<3e-3] = 0
thetas = normalize(thetas,axis=1,norm='l1')
thetas_theoretical = doc_topics_all[0][0:n_docs]

# Eval betas and thetas
eval_betas(beta, topic_vectors)
eval_thetas(thetas_theoretical, thetas, n_docs)

(10000, 50)
Tópicos (equivalentes) evaluados correctamente:
37.92278644385276
Difference in evaluation of doc similarity:
3712.4497197252617


#### 2.1.1. Inference on the centralized approach with validation corpus generated with the same GP

In [19]:
# Define inference corpus
corpus_inf = [doc for docs_node in documents_all for doc in docs_node[n_docs:(n_docs+n_docs_inf)]]
print(len(corpus_inf))

# Perform inference
extractPipe(modeldir)
inference(modeldir, corpus_inf)

50000
-- Extracting pipeline
-- Running command /export/usuarios_ml4ds/jarenas/github/IntelComp/ITMT/topicmodeler/src/topicmodeling/mallet-2.0.8/bin/mallet import-file --use-pipe-from /export/usuarios_ml4ds/lbartolome/data/project_folder/TMmodels/Federated_test/mallet_centralized/corpus.mallet --input /export/usuarios_ml4ds/lbartolome/data/project_folder/TMmodels/Federated_test/mallet_centralized/corpus_aux.txt --output /export/usuarios_ml4ds/lbartolome/data/project_folder/TMmodels/Federated_test/mallet_centralized/import.pipe
-- Inference: Mallet Data Import
-- Running command /export/usuarios_ml4ds/jarenas/github/IntelComp/ITMT/topicmodeler/src/topicmodeling/mallet-2.0.8/bin/mallet import-file --use-pipe-from /export/usuarios_ml4ds/lbartolome/data/project_folder/TMmodels/Federated_test/mallet_centralized/import.pipe --input /export/usuarios_ml4ds/lbartolome/data/project_folder/TMmodels/Federated_test/mallet_centralized/corpus_inf.txt --output /export/usuarios_ml4ds/lbartolome/data/pr

In [20]:
# Get inferred thetas
thetas_inf = np.loadtxt(modeldir.joinpath('doc-topics-inf.txt').resolve().as_posix(), delimiter='\t', dtype=np.float32)[:,2:][:n_docs,:]
thetas_inf[thetas_inf<3e-3] = 0
thetas_inf = normalize(thetas_inf,axis=1,norm='l1')
thetas_theoretical = doc_topics_all[0][n_docs:(n_docs+n_docs_inf)]

# Eval thetas
eval_thetas(thetas_theoretical, thetas_inf, n_docs_inf)

Difference in evaluation of doc similarity:
3738.790479287985


#### 2.1.2. Inference on the centralized approach with validation corpus generated with different GP

In [21]:
# Define inference corpus
corpus_inf = [doc for docs_node in documents_all2 for doc in docs_node]
print(len(corpus_inf))

# Perform inference
extractPipe(modeldir)
inference(modeldir, corpus_inf)

50000
-- Extracting pipeline
-- Running command /export/usuarios_ml4ds/jarenas/github/IntelComp/ITMT/topicmodeler/src/topicmodeling/mallet-2.0.8/bin/mallet import-file --use-pipe-from /export/usuarios_ml4ds/lbartolome/data/project_folder/TMmodels/Federated_test/mallet_centralized/corpus.mallet --input /export/usuarios_ml4ds/lbartolome/data/project_folder/TMmodels/Federated_test/mallet_centralized/corpus_aux.txt --output /export/usuarios_ml4ds/lbartolome/data/project_folder/TMmodels/Federated_test/mallet_centralized/import.pipe
-- Inference: Mallet Data Import
-- Running command /export/usuarios_ml4ds/jarenas/github/IntelComp/ITMT/topicmodeler/src/topicmodeling/mallet-2.0.8/bin/mallet import-file --use-pipe-from /export/usuarios_ml4ds/lbartolome/data/project_folder/TMmodels/Federated_test/mallet_centralized/import.pipe --input /export/usuarios_ml4ds/lbartolome/data/project_folder/TMmodels/Federated_test/mallet_centralized/corpus_inf.txt --output /export/usuarios_ml4ds/lbartolome/data/pr

In [22]:
# Get inferred thetas
thetas_inf = np.loadtxt(modeldir.joinpath('doc-topics-inf.txt').resolve().as_posix(), delimiter='\t', dtype=np.float32)[:,2:][:n_docs,:]
thetas_inf[thetas_inf<3e-3] = 0
thetas_inf = normalize(thetas_inf,axis=1,norm='l1')
thetas_theoretical = doc_topics_all2[0][0:n_docs_inf]

# Eval thetas
eval_thetas(thetas_theoretical, thetas_inf, n_docs_inf)

Difference in evaluation of doc similarity:
3726.1233980466814


### *2.2. Just in one node approach*

In [23]:
# Define corpus
corpus = documents_all[0][0:n_docs]
print(len(corpus))

# Train model 
modelname = "mallet_node0"
modeldir = train(modelname, modelsdir, corpus)

10000
-- -- Creating backup of existing model in /export/usuarios_ml4ds/lbartolome/data/project_folder/TMmodels/Federated_test/mallet_node0_old
-- -- Running command /export/usuarios_ml4ds/jarenas/github/IntelComp/ITMT/topicmodeler/src/topicmodeling/mallet-2.0.8/bin/mallet import-file --preserve-case --keep-sequence --remove-stopwords --token-regex "[\p{L}\p{N}][\p{L}\p{N}\p{P}]*" --input /export/usuarios_ml4ds/lbartolome/data/project_folder/TMmodels/Federated_test/mallet_node0/corpus.txt --output /export/usuarios_ml4ds/lbartolome/data/project_folder/TMmodels/Federated_test/mallet_node0/corpus.mallet
/export/usuarios_ml4ds/jarenas/github/IntelComp/ITMT/topicmodeler/src/topicmodeling/mallet-2.0.8/bin/mallet train-topics --config /export/usuarios_ml4ds/lbartolome/data/project_folder/TMmodels/Federated_test/mallet_node0/trainconfig.json
-- -- Training mallet topic model. Command is /export/usuarios_ml4ds/jarenas/github/IntelComp/ITMT/topicmodeler/src/topicmodeling/mallet-2.0.8/bin/mallet 

Mallet LDA: 50 topics, 6 topic bits, 111111 topic mask
Data loaded.
max tokens: 249
total tokens: 1991693
<10> LL/token: -9.11713
<20> LL/token: -8.35992
<30> LL/token: -8.10539
<40> LL/token: -7.96454

0	0.02	wd968 wd1833 wd112 wd1347 wd2049 wd2635 wd3878 wd803 wd2180 wd1659 wd842 wd3408 wd3539 wd3685 wd2305 wd4755 wd4099 wd4411 wd3667 wd3287 
1	0.02	wd1023 wd4832 wd4442 wd220 wd4712 wd4452 wd2940 wd77 wd4078 wd230 wd4916 wd3955 wd1783 wd2771 wd2305 wd2304 wd3420 wd2438 wd1900 wd1085 
2	0.02	wd3994 wd4760 wd2680 wd4531 wd3495 wd2822 wd391 wd3955 wd1745 wd2341 wd942 wd2780 wd1103 wd4864 wd2910 wd1969 wd3461 wd3996 wd640 wd1833 
3	0.02	wd1735 wd3994 wd1267 wd112 wd1428 wd4830 wd920 wd2680 wd4760 wd4531 wd2341 wd942 wd3554 wd3984 wd2822 wd3455 wd1103 wd1745 wd2780 wd1969 
4	0.02	wd230 wd1694 wd713 wd2037 wd1351 wd3194 wd3000 wd1422 wd1782 wd2468 wd1728 wd4723 wd2620 wd2918 wd2329 wd2240 wd2370 wd2903 wd2721 wd2288 
5	0.02	wd2169 wd3278 wd4397 wd3994 wd2765 wd968 wd4000 wd2049 wd2959 wd13

In [24]:
# Get betas
beta = np.zeros((tm_settings["n_topics"], tm_settings["vocab_size"]))
with open(modeldir.joinpath('word-topic-counts.txt').resolve().as_posix(), 'r', encoding='utf8') as fin:
    for line in fin.readlines():
        tokens = line.split()[1:]
        pos = int(tokens[0][2:])
        for el in tokens[1:]:
            tpc = int(el.split(':')[0])
            cnt = int(el.split(':')[1])
            beta[tpc,pos] = cnt

beta = normalize(beta,axis=1,norm='l1')

# Get thetas
thetas = np.loadtxt(modeldir.joinpath('doc-topics.txt').resolve().as_posix(), delimiter='\t', dtype=np.float32)[:,2:][:n_docs,:]

thetas[thetas<3e-3] = 0
thetas = normalize(thetas,axis=1,norm='l1')
thetas_theoretical = doc_topics_all[0][0:n_docs]
print(thetas_theoretical.shape)

# Eval betas and thetas
eval_betas(beta, topic_vectors)
eval_thetas(thetas_theoretical, thetas, n_docs)

(10000, 50)
Tópicos (equivalentes) evaluados correctamente:
16.810857478486273
Difference in evaluation of doc similarity:
2425.526791857555


#### 2.2.1. Inference on the just in one node approach with validation corpus generated with the same GP

In [25]:
# Define inference corpus
corpus_inf = documents_all[0][n_docs:(n_docs+n_docs_inf)]
print(len(corpus_inf))

# Perform inference
extractPipe(modeldir)
inference(modeldir, corpus_inf)

10000
-- Extracting pipeline
-- Running command /export/usuarios_ml4ds/jarenas/github/IntelComp/ITMT/topicmodeler/src/topicmodeling/mallet-2.0.8/bin/mallet import-file --use-pipe-from /export/usuarios_ml4ds/lbartolome/data/project_folder/TMmodels/Federated_test/mallet_node0/corpus.mallet --input /export/usuarios_ml4ds/lbartolome/data/project_folder/TMmodels/Federated_test/mallet_node0/corpus_aux.txt --output /export/usuarios_ml4ds/lbartolome/data/project_folder/TMmodels/Federated_test/mallet_node0/import.pipe
-- Inference: Mallet Data Import
-- Running command /export/usuarios_ml4ds/jarenas/github/IntelComp/ITMT/topicmodeler/src/topicmodeling/mallet-2.0.8/bin/mallet import-file --use-pipe-from /export/usuarios_ml4ds/lbartolome/data/project_folder/TMmodels/Federated_test/mallet_node0/import.pipe --input /export/usuarios_ml4ds/lbartolome/data/project_folder/TMmodels/Federated_test/mallet_node0/corpus_inf.txt --output /export/usuarios_ml4ds/lbartolome/data/project_folder/TMmodels/Federate

In [26]:
# Get inferred thetas
thetas_inf = np.loadtxt(modeldir.joinpath('doc-topics-inf.txt').resolve().as_posix(), delimiter='\t', dtype=np.float32)[:,2:][:n_docs,:]
thetas_inf[thetas_inf<3e-3] = 0
thetas_inf = normalize(thetas_inf,axis=1,norm='l1')
thetas_theoretical = doc_topics_all[0][n_docs:(n_docs+n_docs_inf)]

# Eval thetas
eval_thetas(thetas_theoretical, thetas_inf, n_docs_inf)

Difference in evaluation of doc similarity:
2451.006825123292


#### 2.2.2. Inference on the just in one node approach with validation corpus generated with different GP

In [27]:
# Define inference corpus
corpus_inf = documents_all2[0][0:n_docs]
print(len(corpus_inf))

# Perform inference
extractPipe(modeldir)
inference(modeldir, corpus_inf)

10000
-- Extracting pipeline
-- Running command /export/usuarios_ml4ds/jarenas/github/IntelComp/ITMT/topicmodeler/src/topicmodeling/mallet-2.0.8/bin/mallet import-file --use-pipe-from /export/usuarios_ml4ds/lbartolome/data/project_folder/TMmodels/Federated_test/mallet_node0/corpus.mallet --input /export/usuarios_ml4ds/lbartolome/data/project_folder/TMmodels/Federated_test/mallet_node0/corpus_aux.txt --output /export/usuarios_ml4ds/lbartolome/data/project_folder/TMmodels/Federated_test/mallet_node0/import.pipe
-- Inference: Mallet Data Import
-- Running command /export/usuarios_ml4ds/jarenas/github/IntelComp/ITMT/topicmodeler/src/topicmodeling/mallet-2.0.8/bin/mallet import-file --use-pipe-from /export/usuarios_ml4ds/lbartolome/data/project_folder/TMmodels/Federated_test/mallet_node0/import.pipe --input /export/usuarios_ml4ds/lbartolome/data/project_folder/TMmodels/Federated_test/mallet_node0/corpus_inf.txt --output /export/usuarios_ml4ds/lbartolome/data/project_folder/TMmodels/Federate

In [28]:
# Get inferred thetas
thetas_inf = np.loadtxt(modeldir.joinpath('doc-topics-inf.txt').resolve().as_posix(), delimiter='\t', dtype=np.float32)[:,2:][:n_docs,:]
thetas_inf[thetas_inf<3e-3] = 0
thetas_inf = normalize(thetas_inf,axis=1,norm='l1')
thetas_theoretical = doc_topics_all2[0]

# Eval thetas
eval_thetas(thetas_theoretical, thetas_inf, n_docs_inf)

Difference in evaluation of doc similarity:
2438.488190789704


## **3. ProdLDA**


---

In [29]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from scipy import sparse

In [30]:
cd /export/usuarios_ml4ds/lbartolome/topicmodeler

/export/usuarios_ml4ds/lbartolome/topicmodeler


In [31]:
from src.topicmodeling.neural_models.pytorchavitm.datasets.bow_dataset import BOWDataset
from src.topicmodeling.neural_models.pytorchavitm.avitm_network.avitm import AVITM

### *Auxiliary functions*

In [32]:
def convert_topic_word_to_init_size(vocab_size, model, model_type,
                                    ntopics, id2token, all_words):
    """It converts the topic-word distribution matrix obtained from the training of a model into a matrix with the dimensions of the original topic-word distribution, assigning zeros to those words that are not present in the corpus. 
    It is only of use in case we are training a model over a synthetic dataset, so as to later compare the performance of the attained model in what regards to the similarity between the original and the trained model.

    Args:
        * vocab_size (int):       Size of the synethic'data vocabulary.
        * model (AVITM/CTM):      Model whose topic-word matrix is being transformed.
        * model_type (str):       Type of the trained model (e.g. AVITM)
        * ntopics (int):          Number of topics of the trained model.
        * id2token (List[tuple]): Mappings with the content of the document-term matrix.
        * all_words (List[str]):  List of all the words of the vocabulary of size vocab_size.

    Returns:
        * ndarray: Normalized transormed topic-word distribution.
    """
    if model_type == "avitm":
        w_t_distrib = np.zeros((ntopics, vocab_size), dtype=np.float64)
        wd = model.get_topic_word_distribution()
        for i in np.arange(ntopics):
            for idx, word in id2token.items():
                for j in np.arange(len(all_words)):
                    if all_words[j] == word:
                        w_t_distrib[i, j] = wd[i][idx]
                        break
        normalized_array = normalize(w_t_distrib,axis=1,norm='l1')
        return normalized_array
    else:
        print("Method not impleemnted for the selected model type")
        return None

In [33]:
def train_avitm(modelname, modelsdir, corpus):
    
    # Create model folder
    modeldir, configFile = create_model_folder(modelname, modelsdir)
    
    # Create corpus in ProdLDA format (BoWDataset)
    cv = CountVectorizer(input='content', lowercase=True, stop_words='english', binary=False)
    docs = [" ".join(corpus[i]) for i in np.arange(len(corpus))]
    train_bow = cv.fit_transform(docs).toarray()
    idx2token = cv.get_feature_names_out()
    train_dataset = BOWDataset(train_bow, idx2token)
    input_size = len(idx2token)
    id2token = {k: v for k, v in zip(range(0, len(idx2token)), idx2token)}
    
    avitm = AVITM(logger=None,
              input_size=input_size,
              n_components=n_topics,
              model_type="prodLDA",
              hidden_sizes=(100, 100),
              activation='softplus',
              dropout=0.2,
              learn_priors=True,
              batch_size=64,
              lr=2e-3,
              momentum=0.99,
              solver='adam',
              num_epochs=100,
              reduce_on_plateau=False,
              topic_prior_mean=0.0,
              topic_prior_variance=None,
              num_samples=10,
              num_data_loader_workers=0,
              verbose=True)
    avitm.fit(train_dataset)
    
    return modeldir, avitm, cv, id2token, idx2token

### *3.1. Centralized approach*

In [34]:
# Define corpus
corpus = [doc for docs_node in documents_all for doc in docs_node[0:n_docs]]
print(len(corpus))

# Train model 
modelname = "prod_centralized"
modeldir, avitm, cv, id2token, idx2token = train_avitm(modelname, modelsdir, corpus)

50000
-- -- Creating backup of existing model in /export/usuarios_ml4ds/lbartolome/data/project_folder/TMmodels/Federated_test/prod_centralized_old
Settings: 
                N Components: 50
                Topic Prior Mean: 0.0
                Topic Prior Variance: None
                Model Type: prodLDA
                Hidden Sizes: (100, 100)
                Activation: softplus
                Dropout: 0.2
                Learn Priors: True
                Learning Rate: 0.002
                Momentum: 0.99
                Reduce On Plateau: False
                Save Dir: None


Epoch: [100/100]	 Seen Samples: [5000000/5000000]	Train Loss: 1569.045916875	Time: 0:00:09.622405: : 100it [15:01,  9.02s/it]  
Sampling: [20/20]: : 20it [01:48,  5.44s/it]


In [35]:
# Get betas
betas = avitm.get_topic_word_distribution()
all_words = ['wd'+str(word) for word in np.arange(vocab_size+1) if word > 0]
betas = convert_topic_word_to_init_size(vocab_size=vocab_size,
                                        model=avitm,
                                        model_type="avitm",
                                        ntopics=n_topics,
                                        id2token=id2token,
                                        all_words=all_words)

# Get thetas
thetas = np.asarray(avitm.get_doc_topic_distribution(avitm.train_data))[0:n_docs,:]
thetas[thetas < 3e-3] = 0
thetas = normalize(thetas, axis=1, norm='l1')
thetas_theoretical = doc_topics_all[0][0:n_docs]

# Eval betas and thetas
eval_betas(beta, topic_vectors)
eval_thetas(thetas_theoretical, thetas, n_docs)

Sampling: [20/20]: : 20it [01:49,  5.45s/it]


Tópicos (equivalentes) evaluados correctamente:
16.810857478486273
Difference in evaluation of doc similarity:
678.6998611282362


#### 3.1.1. Inference on the centralized approach with validation corpus generated with the same GP

In [36]:
# Define inference corpus
corpus_inf = [doc for docs_node in documents_all for doc in docs_node[n_docs:(n_docs+n_docs_inf)]]
print(len(corpus_inf))


# Get inferred thetas
docs_val_conv = [" ".join(corpus_inf[i]) for i in np.arange(len(corpus_inf))]
val_bow = cv.transform(docs_val_conv)
val_bow = val_bow.toarray()
val_data = BOWDataset(val_bow, idx2token)

thetas_inf = np.asarray(avitm.get_doc_topic_distribution(val_data))[0:n_docs,:]
thetas_inf[thetas_inf < 3e-3] = 0
thetas_inf = normalize(thetas_inf, axis=1, norm='l1')

thetas_theoretical = doc_topics_all[0][n_docs:(n_docs+n_docs_inf)]

# Eval thetas
eval_thetas(thetas_theoretical, thetas_inf, n_docs_inf)

50000


Sampling: [20/20]: : 20it [01:50,  5.52s/it]


Difference in evaluation of doc similarity:
879.5030616941613


#### 3.1.2. Inference on the centralized approach with validation corpus generated with different GP

In [37]:
# Define inference corpus
corpus_inf = [doc for docs_node in documents_all2 for doc in docs_node]
print(len(corpus_inf))

# Get inferred thetas
docs_val_conv = [" ".join(corpus_inf[i]) for i in np.arange(len(corpus_inf))]
val_bow = cv.transform(docs_val_conv)
val_bow = val_bow.toarray()
val_data = BOWDataset(val_bow, idx2token)

thetas_inf = np.asarray(avitm.get_doc_topic_distribution(val_data))[0:n_docs,:]
thetas_inf[thetas_inf < 3e-3] = 0
thetas_inf = normalize(thetas_inf, axis=1, norm='l1')

thetas_theoretical = doc_topics_all2[0][0:n_docs_inf]

# Eval thetas
eval_thetas(thetas_theoretical, thetas_inf, n_docs_inf)

50000


Sampling: [20/20]: : 20it [01:50,  5.54s/it]


Difference in evaluation of doc similarity:
889.3077317589997


### *3.2. Just in one node approach*

In [38]:
# Define corpus
corpus = documents_all[0][0:n_docs]
print(len(corpus))

# Train model 
modelname = "prodlda_node0"
modeldir, avitm, cv, id2token, idx2token = train_avitm(modelname, modelsdir, corpus)

# Get betas
betas = avitm.get_topic_word_distribution()
all_words = ['wd'+str(word) for word in np.arange(vocab_size+1) if word > 0]
betas = convert_topic_word_to_init_size(vocab_size=vocab_size,
                                        model=avitm,
                                        model_type="avitm",
                                        ntopics=n_topics,
                                        id2token=id2token,
                                        all_words=all_words)

# Get thetas
thetas = np.asarray(avitm.get_doc_topic_distribution(avitm.train_data))
thetas[thetas < 3e-3] = 0
thetas = normalize(thetas, axis=1, norm='l1')
thetas_theoretical = doc_topics_all[0][0:n_docs]

# Eval betas and thetas
eval_betas(beta, topic_vectors)
eval_thetas(thetas_theoretical, thetas, n_docs)

10000
-- -- Creating backup of existing model in /export/usuarios_ml4ds/lbartolome/data/project_folder/TMmodels/Federated_test/prodlda_node0_old
Settings: 
                N Components: 50
                Topic Prior Mean: 0.0
                Topic Prior Variance: None
                Model Type: prodLDA
                Hidden Sizes: (100, 100)
                Activation: softplus
                Dropout: 0.2
                Learn Priors: True
                Learning Rate: 0.002
                Momentum: 0.99
                Reduce On Plateau: False
                Save Dir: None


Epoch: [100/100]	 Seen Samples: [1000000/1000000]	Train Loss: 1566.028729296875	Time: 0:00:03.645799: : 100it [06:15,  3.76s/it]
Sampling: [20/20]: : 20it [01:02,  3.13s/it]
Sampling: [20/20]: : 20it [01:03,  3.16s/it]


Tópicos (equivalentes) evaluados correctamente:
16.810857478486273
Difference in evaluation of doc similarity:
587.7841887872687


#### 3.2.1. Inference on the just in one node approach with validation corpus generated with the same GP

In [39]:
# Define inference corpus
corpus_inf = documents_all[0][n_docs:(n_docs+n_docs_inf)]
print(len(corpus_inf))

# Get inferred thetas
docs_val_conv = [" ".join(corpus_inf[i]) for i in np.arange(len(corpus_inf))]
val_bow = cv.transform(docs_val_conv)
val_bow = val_bow.toarray()
val_data = BOWDataset(val_bow, idx2token)

thetas_inf = np.asarray(avitm.get_doc_topic_distribution(val_data))
thetas_inf[thetas_inf < 3e-3] = 0
thetas_inf = normalize(thetas_inf, axis=1, norm='l1')

thetas_theoretical = doc_topics_all[0][n_docs:(n_docs+n_docs_inf)]

# Eval thetas
eval_thetas(thetas_theoretical, thetas_inf, n_docs_inf)

10000


Sampling: [20/20]: : 20it [01:00,  3.03s/it]


Difference in evaluation of doc similarity:
910.3343044933794


#### 2.2.2. Inference on the just in one node approach with validation corpus generated with different GP

In [40]:
# Define inference corpus
corpus_inf = documents_all2[0][0:n_docs]
print(len(corpus_inf))

# Get inferred thetas
docs_val_conv = [" ".join(corpus_inf[i]) for i in np.arange(len(corpus_inf))]
val_bow = cv.transform(docs_val_conv)
val_bow = val_bow.toarray()
val_data = BOWDataset(val_bow, idx2token)

thetas_inf = np.asarray(avitm.get_doc_topic_distribution(val_data))
thetas_inf[thetas_inf < 3e-3] = 0
thetas_inf = normalize(thetas_inf, axis=1, norm='l1')

thetas_theoretical = doc_topics_all2[0]

# Eval thetas
eval_thetas(thetas_theoretical, thetas_inf, n_docs_inf)

10000


Sampling: [20/20]: : 20it [01:00,  3.00s/it]


Difference in evaluation of doc similarity:
916.3224718630262
